---
title: "Chapter Two: Clustering the Council"
format:
  html: default
  pdf:
    df-print: kable
    tbl-colwidths: [75,25]
    code-fold: true
    mainfont: "Georgia"
header-includes:
  - \usepackage{graphicx}
  - \usepackage{adjustbox}
  - \usepackage{afterpage}
  - \usepackage{biblatex}
  - \newcommand{\bibsection}{\clearpage\printbibliography}
execute:
  echo: false
jupyter: python3
bibliography: references.bib
---

```{r libraries, echo=FALSE,warning=FALSE, message=FALSE}
suppressPackageStartupMessages(library(tidyverse))
library(rio)
suppressPackageStartupMessages(library(flextable))
library(gt)
library(RColorBrewer)
library(sf)
library(stargazer)
library(modelsummary)
library(fixest)
```

```{r data, echo=FALSE,warning=FALSE, message=FALSE}
meta_clusters <- read_csv('../../member_chapters/data/meta_cluster_March17.csv')
bsr <- read_csv('../../cc_legislation/data/bills_with_cluster_count_MARCH24.csv')
tf_idf<- read_csv("../data/top_words_by_cluster.csv")
cc_sf <- read_sf('../../neighborhoods_ccdistricts/data/shapefiles/city_council')
c3_results <- read_csv("../../cc_election_cleaning/election_results_with_vote_sponsor_cluster_DEC04.csv")
new_clusters <- read_csv("../data/electoral_results_with_new_cluster.csv")
demo_clusters <- read_csv("../../cc_election_cleaning/district_level_demo_clusters.csv")
cluster_influence <- read_csv("../../cc_election_cleaning/top_variables_with_scores_per_cluster.csv")

cluster_three <- c3_results %>%
  filter(kmode_cluster == 3)

cluster_map<-winners%>%
  select(district,new_cluster)%>%
  distinct()
demo_clusters <- left_join(demo_clusters,cluster_map,by="district")

```

```{r flex_function, include=FALSE}
FitFlextableToPage <- function(ft, pgwidth = 6){

 
# set as autofit to make width parameters adjustable

ft_out <- ft %>% autofit()


# set width as function of page width

ft_out <- width(ft_out, width = dim(ft_out)$widths*pgwidth /(flextable_dim(ft_out)$widths))


return(ft_out)
}
```


This chapter will explore empirically the theoretical schema laid out in the previous chapter. Specifically, it will break the New York City Council into ideological factions that can then be analyzed as discrete political units. To do so it will use council members’ voting records, bill sponsorship, campaign finance, endorsements, expert analysis, and caucus membership. The task is complicated by the fact that most council votes are lopsided and by the high rate of co-sponsorship, every member has co-sponsored with another at least once. Nonetheless, clear factions do emerge. First and most clearly a conservative faction, then a far-left DSA centered faction, then more moderate left factions. Finally, demographic support for these factions is discussed in the context of the discussion of social-democratic party families in Chapter 1. It finds specific demographic patterns of support for these progressive candidates, specifically the number of non-Hispanic whites born outside of New York State and the share of residents employed as journalists, artists, and educators. This effect is particularly pronounced for DSA candidates. It then further divides the progressives by the demographic characteristics of their district. Of the six demographic categories in the city, the progressives succeed in three of them. It will explore what demographic patterns of support emerge inside those three categories. It will conclude that these three categories offer a blueprint for progressive coalition formation, pointing towards the next chapters in which induvial races will be explored in more detail.

Throughout this chapter K-Means and K-Mode clustering will be used to break both council members and council districts into smaller like units. The mechanics of these two methods are explained in more detail below, but in general they sort a larger group into a pre-determined number of smaller groups by how similar they are on a set of categories, e.g. voting record, co-sponsorship, etc. They are different from other methods of dimensionality reduction often used in the social-sciences, such as factor analysis and principal component analysis, in that they are simply arranging datapoints into clusters rather than attempting to detect a latent explanatory variable.

## Voting

Council floor votes are heavily lopsided, most pass with only a few dissenting conservative voices. This makes isolating the Council's conservatives quite easy but makes breaking the rest of the body into clusters more difficult. To do this K-Means clustering, a clustering methodology with wide application which seeks to partition a set of observations into a pre-defined number of clusters around centroids that minimize within cluster variance, was employed [@lloyd1982a; @steinley2006a]. K‑Means starts by choosing a fixed number of “group centers” (or centroids) and then assigns each council member to whichever center is closest to their overall voting record. It next recalculates each center as the average voting profile of its assigned members, and then re‑assigns members to the nearest updated center—repeating these two steps until the groups stabilize. By one‑hot coding every “yes,” “no,” or “abstain” vote, each vote is turned into a separate binary feature so that the algorithm can measure similarity across all bills. Finally, simple diagnostics (for example, looking for the “elbow” in a plot of within‑group variance or checking silhouette scores) are employed to pick three clusters that balance clarity and cohesion. The data used to cluster were all floor votes from 2022 and 2023, reduced to the 100 closest votes with at least 30 members voting, measured by the ratio of anti-votes and abstentions to pro votes. No matter how many clusters were defined, the method pulls out the conservatives, who vote no or abstain as a bloc on much legislation. If more than three clusters are defined, it also pulls out a group of progressive council members including both DSA members as well as other progressive stalwarts such as Chi Ossé, Jennifer Guttierez, Sandy Nurse, and Shahana Hanif, several of which were mentioned in interviews with DSA members as close to the party. Table 1 below shows both the Conservative and Progressive bloc.


```{r,table 1,echo=FALSE,warning=FALSE , results='asis'}
meta_clusters %>%
  select(proper_name, vote_cluster) %>%
  group_by(vote_cluster) %>%
  summarize(proper_names = paste(proper_name, collapse = ", "))%>%
  gt()%>%
  cols_label(vote_cluster = 'Cluster',proper_names = 'Council Members')
```


K-Means clustering performed with the Python library Sickit-Learn allows for examination of how each item on which the clustering was performed (here, votes on bills) affected the overall cluster placement.[^1] The bills that most influenced the placement of these progressive members into a cluster are no votes on budget bills and mayoral appointments (all of which nonetheless passed). This aligns with DSA's members insistence on the primacy of budget and the importance of voting against what they condier to be austerity budgets. No matter how many more clusters are specified no new group cluster forms; individuals from the massive non-progressive or conservative cluster start to peel off one by one. There are therefore three voting clusters in the Council: a large majority that vote yes on most bills for which they are present, and two dissenting minorities, a conservative one who votes no quite frequently and a progressive one who votes no on key pieces of legislation, especially in cases in which to do so is antagonistic to the mayor. It is the latter that is of the most interest to this project as it is a demonstration of the vying for regime control described in Chapter 1. This portion of the council, loud and active in its opposition to Mayor Adams's conservative policy agenda, has helped push the entire body more firmly into its role of Mayoral oversight, culminating in overridden vetoes and the Speaker referring to the body as a "co-equal" branch of government [@adams].

[^1]: "Scikit-learn: Machine Learning in Python," Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

## Co-Sponsorship

Examining the patterns in which members sponsor bills also faces challenges, though it also reveals similar groups of conservative and progressive legislators. Many bills, especially those that pass, have high numbers of sponsors. The mean number of sponsors for a bill in the dataset was 13, with the mean number for a bill that passed at 23. Additionally, there is a lot of co-sponsorship across ideological lines; every member has co-sponsored with every other member at least once, often many more times than once. This is perhaps not surprising, as Lincoln Restler highlighted, voters prize pragmatism in their council votes and members are aware of this, so there is motivation to work across boundaries to get things done [@restler2024a]. This high rate of cooperation means that using traditional methods to visualize networks leads to a large bird's nest of co-sponsorship, a version of this tangled graph is included in the appendix. Nonetheless, members do co-sponsor with some members much more frequently than others, so despite the significant noise caused by cooperation patterns do emerge.

To identify these patterns, K-Means clustering was once again employed. To prepare the data, a matrix that described how often each member co-sponsored with another on a scale of 0-1 was formed, with 1 representing co-sponsorship on every single bill. Bills that had 40 or more co-sponsors were removed to reduce noise and isolate more meaningful co-sponsorships. The K-Means was then run on this matrix. The results of this are recorded in Table 2 below. The results mirror those found in the voting patterns above, cluster 1 again features both DSA members as well as other (though not all) members from the voting clusters above. This cluster remained stable as long as there were more than two clusters defined.


```{r,table 2,echo=FALSE,warning=FALSE , results='asis'}

meta_clusters %>%
  select(proper_name, k_spon) %>%
  group_by(k_spon) %>%
  summarize(proper_names = paste(proper_name, collapse = ", "))%>%
  gt()%>%
  cols_label(k_spon = 'Cluster',proper_names = 'Council Members')

```


The Louvain method of community detection was used to check the robustness of the K-Means results. This algorithm takes data that has already been structured for network mapping (a different form than the matrix used for the K-Means) and detects communities based on the weights of the edges (connections) in this network. Here clusters are not predefined, the algorithm arrives at a number on its own. This method of community detection has seen increasing application in Political Science research, including a similar task of detecting small and nuanced communities in co-sponsorship of bills in the UN General Assembly [@meyer2021]. The results are displayed in Table 3 below, where the the Louvain method picks up a similar group of progressives, though with additions, to the group formed by the K-Means.


```{r,table 3,echo=FALSE,warning=FALSE , results='asis'}
ft <- meta_clusters %>%
  select(proper_name, louv) %>%
  group_by(louv) %>%
  summarize(proper_names = paste(proper_name, collapse = ", "))%>%
  flextable()%>%
  autofit()

ft
```


## Sponsored Bills

The bills these progressives co-sponsor fall into two general categories, bills that attract a high-rate of fellow progressives and have a low chance of passing, and bills that are primarily sponsored by progressives but attract a wider level of support and therefore have a higher chance of passing. Both groups of bills track closely with the egalitarian urban order defined by Weaver and discussed in Chapter 1. Table 5 shows the bills which have the highest percentage of sponsors from the progressive cluster. They concern worker protections, oversight of the NYPD, tenant advocacy, environmental concerns, college counseling for low-income New Yorkers and protections for cyclists. Not a single bill with more than 60% of its sponsors from this cluster was enacted, so while these progressive-heavy bills may be pushing the rhetoric of the council left they have not, as of yet, translated into legislation.


```{r, table 5,echo=FALSE,warning=FALSE , results='asis'}
bsr %>%   select(EventItemMatterName, percent_cluster_4) %>%   arrange(desc(percent_cluster_4)) %>%   slice(1:10)%>%   gt()%>%   cols_label(EventItemMatterName = 'Bill Summary',percent_cluster_4 = '% of Sponsors from Progressive Cluster')
```


This does not mean, however, that the group is ineffective. Dozens of bills sponsored by members of this progressive cluster have been enacted on topics that are slightly different but similarly progressive. Table 6 below shows the enacted cluster 3 bills with the highest rate of co-sponsorship, a full list of bills they have sponsored and passed are available in the appendix. These bills have to do with worker protections, migrant rights, the distribution of free menstrual products, the protections of LGBTQ New Yorkers, the protection of reproduction rights, protections for the homeless, and a "Marshall Plan for Moms" to help support working mothers. These members have clearly set a progressive agenda both in practice and in rhetoric. To compare these bills to those sponsored by other clusters table 7 shows a Term Frequency -- Inverse Document Frequency (TF-IDF) of the text descriptions of the bills sponsored by each group. What this in effect shows are the words that occur most frequently in each group's legislation but less in others.


```{r, table 6,echo=FALSE,warning=FALSE , results='asis'}
bsr %>%   
filter(primary_sponsor_cluster == 3 & EventItemMatterStatus == 'Enacted')%>%   select(EventItemMatterName, percent_cluster_3) %>%   arrange(desc(percent_cluster_3)) %>%   slice(1:10)%>%   gt()%>% cols_label(EventItemMatterName = 'Bill Summary',percent_cluster_3 = '% of Sponsors from Cluster 3')
```


</br>


```{r, table 7,echo=FALSE,warning=FALSE , results='asis'}
ft<- tf_idf%>%  
  flextable()  
ft
```


While both the voting data and sponsorship is somewhat difficult to parse due to the rate of cooperation and lopsided voting, trends do emerge from each that complement the other. In the voting data, a group of progressive council members emerge as a group of protest voters, seeking to check or register objection to Mayor Adams's more conservative impulses. This same group of protest votes sponsor bills together, and these sponsorships fall into two categories 1) bills with high rates of progressive co-sponsorship that often deal with progressive wish lists and 2) bills with progressive members as the prime sponsors that gain higher rates of other cluster sponsors and have a much higher rate of passing. All of this activity fits the model of a progressive core seeking to counter a conservative mayor's agenda and establish a progressive regime. The next section will examine if these trends in voting and sponsorship are also mirrored in electoral patterns.

## Meta Clustering

Despite some variation in its specific makeup, both methods agree that there is a progressive bloc sponsoring together and voting in protest together. To try and make sense of these similar but slightly different clustering blocs, other important variables are introduced and a process of meta-clustering was performed. The variables used in addition to the voting and sponsorship clusters are: 1) a catagorical variable that records whether members remained in the progressive caucus after a max exodus over defunding the NYPD, left the caucus, or were never members, 2) how the members were categorized by NYC political journalist Michael Lange, who offers a detailed categorization of ideal type NYC politicians[^2], 3) whether the members were endorsed by the Working Families Party, and 4) a clustering of how their campaign was financed, which looked at the number and amount of individual contributions as well as what percentage of the contributions originated from inside the district[^3]. The finance variable is included despite not being directly tied to governance because of a theorized relationship between how campaigns are financed and the political valence of the candidate. In NYC further left candidates have tended to receive higher rates of low-dollar donations from all over the city, with significant contributions coming from out of state, with a high percentage of their overall campaign contributions coming from individual donors.

[^2]: Lange explores these categorizations in a viral substack post [@lange2022], he expanded on this and offered a categorization of every member of the body in an interview conducted for this dissertation [@lange2025].

[^3]: The process of clustering was identical to the one performed on voting and sponsorship. A table is available in the appendix.

This dataset features all categorical data in which, for most variables, the specific number had no meaning. In order to cluster based on these individual categories K-Mode clustering was employed. K‑Mode works much like K‑Means but swaps out averages for most common categories. First a fixed number of prototype category‑patterns (modes) is selected. Then each observation is assigned to the mode it matches most closely; that is, the one with which it has the fewest category‐mismatches. Next, each mode is recalculated by taking, for each variable, the single category that appears most often among its assigned observations. These two steps—assigning to the nearest mode and then updating the modes—repeat until the groups stop changing. Finally, again like K-Means, simple diagnostics (for example, looking for a sharp drop in total mismatches as k increases) to confirm that the chosen number of clusters best balances simplicity and explanatory power. Table 4 shows the group that emerges from this process.


```{r, table 4,echo=FALSE,warning=FALSE , results='asis'}
meta_clusters %>%
  select(proper_name, new_cluster) %>%
  #filter(new_cluster == 4)%>%
  group_by(new_cluster) %>%
  summarize(proper_names = paste(proper_name, collapse = ", "))%>%
  gt()%>%
  cols_label(new_cluster = 'Cluster',proper_names = 'Council Members')
```


Cluster four contains the progressives and passes several common sense tests of accuracy. It includes both DSA endorsed candidates as well as self-proclaimed socialists Shahana Hanif and Chi Osse. Every member in cluster four remained in the progressive caucus after the split over NYPD funding. They all fall primarily into the furthest left categories of Lange’s categorization, the DSA, the alphabet left, and the hybrid progressives. The exception to this latter points is Amanda Farias, who Lange categorized as Center Left Establishment. Lange hesitated on her assignment however, and speaker Adams recently touted making her majority leader as a sign of her progressive bona fides.

## Electoral Patterns

This section will first look at what distinguishes the council districts won by the progressive cluster from other districts, then turn to look at their performance by electoral districts to see what, if any, demographic patterns predicted their success. Subsequent chapters will then examine how these demographic trends hold race by race. Table 8 offers some descriptive statistics on the council districts won by members of the progressive cluster, cluster 3. They are the second whitest cluster, with the highest rate of white transplants, non-Hispanic whites born outside of New York State. They have high education levels and low home ownership levels. Figure 1 maps all the clusters, including only districts that had a contested Democratic primary in 2021.


```{r, table 8,echo=FALSE,warning=FALSE , results='asis',message=FALSE}
winners <- new_clusters%>%
  filter(winner == TRUE)

winners %>%
  drop_na(new_cluster,white_transplant_ratio,nhw21p,nhb21p,cvap21bapp,hh21op)%>%
  group_by(new_cluster)%>%
  summarize(
    perc_white = mean(nhw21p),
    perc_black = mean(nhb21p),
    degree = mean(cvap21bapp),
    homeowners = mean(hh21op),
    wh_tr = mean(white_transplant_ratio),
    income = mean(mhhi21)
  )%>%
  gt()%>%
  cols_label(new_cluster = 'Cluster', perc_white = 'NH White Share',
              perc_black = 'NH Black Share', degree = '% BA or Higher',
              homeowners = '% Homeowners', wh_tr = '% White Transplants')

```


<br/>


```{r figure 1, echo=FALSE, fig.height = 6,fig.width= 8,dpi= 300 }



district_clusters <- new_clusters%>%
  drop_na(new_cluster)%>%
  group_by(district,new_cluster)%>%
  summarise(cluster = mean(new_cluster))
district_clusters$CounDist <- district_clusters$district
cluster_map <- left_join(cc_sf,district_clusters, by = 'CounDist')

ggplot(data = cluster_map) +
  geom_sf(aes(fill = factor(cluster))) + 
  scale_fill_brewer(palette = "Set3", na.value = "gray80") +
  theme_minimal() +
  labs(title = "Council Districts Clustered", 
       fill = "Cluster")
```


Cluster 3's homebase is the corridor from Sunset Park North through Park Slope, passing through a large chunk of North Brooklyn and then into Long Island City and Astoria. A DSA member narrowly lost a race in Brooklyn's 35th district, which would have added to this progressive weight in central Brooklyn. This location, combined with the demographic descriptives included above, suggests that the truism of progressive politics appealing to white, well-educated, economically downwardly mobile, newcomers may be true, though of course this data is not granular enough to conclude this. It at least suggests a geographic base of progressive politics in a diverse and rapidly changing corner of the city and a broad idea of its demographic characteristics. To dig further, this section will now turn to the ED level results for the districts in cluster 3, looking only at the first choice of the Ranked Choice Voting.

To further investigate these demographic variables an OLS model was run on the 919 election districts in which a member of cluster 3 ran, using vote share for those candidates as its dependent variable. The independent variables come from two sources, the tract-level ACS 5-Year file and NYC Open Data, which allows access to datasets from various city agencies. The variables are the log of the median household income, the Black and Hispanic share of an ED, the share of non-Hispanic whites born outside New York State, the share possessing a BA or more, the share of homeowners, and two variables to register noise complaints. One noise variable is the average number of 311 noise complaints per month, the other is the combined number of noise complaints made during the summer of 2020 when there was a large spike in complaints in many areas of the city. The two variables are negatively correlated, implying perhaps summer complaint spikes happened in areas with low average numbers of complaints, so they were both included. Multicollinearity is a concern with these closely related demographic variables, but no variables are correlated at a level higher than .7 and no variable has a VIF higher than 3. The mean and median of all the IVs are listed below.


```{r, table 9,echo=FALSE,warning=FALSE , results='asis'}

prog_cluster <- new_clusters%>%
  filter(new_cluster == 4)
prog_cluster <- prog_cluster %>%
  mutate(log_income = ifelse(mhhi21 > 0, log(mhhi21), NA))
prog_cluster%>%
  summarise(mean_log = mean(log_income),
            med_log = median(log_income),
            mean_black = mean(nhb21p),
            med_black = median(nhb21p),
            mean_hisp = mean(h21p),
            med_hisp = median(h21p),
            mean_wt = mean(white_transplant_ratio),
            med_wt = median(white_transplant_ratio),
            mean_ba = mean(cvap21bapp),
            med_ba = median(cvap21bapp),
            mean_ho = mean(hh21op),
            med_ho = median(hh21op),
            mean_snc = mean(summer_noise_complaints),
            med_snc = median(summer_noise_complaints),
            mean_mn = mean(mean_noise),
            med_mn = median(mean_noise),
            mean_pr = mean(perc_retail),
            med_pr = median(perc_retail),
            )%>%
  gt()

```


The geographic unit in these models is the election precinct, coefficients are measuring the effect of the independent variables on the vote share of the progressive candidates in each of these precincts. The largest significant positive coefficients are for the ratio of white transplants and the share of residents working as journalists and artists, and the largest significant negative coefficient is for income. These results suggest progressive success in neighborhoods with lower incomes, high rates of white transplants, and high rates of those in intellectual, cultural professions. Also of note is the relatively low R squared, only the final model manages to explain more than one quarter of the variance in vote share. One reason for this low explanatory power is the high amount of variation between districts. This model includes districts from four out of five boroughs with a wide range of demographic characteristics. The next section will analyze with more detail the particulars of electoral coalitions in demographically distinct areas of the city.


```{r, table 10,echo=FALSE,warning=FALSE , results='asis'}
prog_cluster<-prog_cluster%>%
  filter(candidate %in% c('Tiffany Caban','Alexa Aviles','Chi A. Osse','Jennifer Gutierrez','Pierina Ana Sanchez','Shahana K. Hanif','Lincoln Restler',
                          'Amanda Farias','Sandy Nurse','Julie Won'))

prog_cluster <- prog_cluster %>%
  mutate(log_income = ifelse(mhhi21 > 0, log(mhhi21), NA))

lm1 <- lm(vote_share ~ log_income + nhb21p, data = prog_cluster, na.action = na.omit)
lm2 <- lm(vote_share ~ log_income + nhb21p + h21p, data = prog_cluster, na.action = na.omit)
lm3 <- lm(vote_share ~ log_income + nhb21p + h21p + white_transplant_ratio, data = prog_cluster, na.action = na.omit)
lm4 <- lm(vote_share ~ log_income + nhb21p + h21p + white_transplant_ratio + cvap21bapp + hh21op + w2admp, data = prog_cluster, na.action = na.omit)
lm5 <- lm(vote_share ~ log_income + nhb21p + h21p + white_transplant_ratio + cvap21bapp + hh21op + w2admp + w2oadp + w2edup + w2pservep, data = prog_cluster, na.action = na.omit)



stargazer(lm1,lm2,lm3,lm4,lm5, type = 'html', covariate.labels=c('Log MHHI', 'NH Black Share','Hispanic Share','White Transplant Share',
'% BA or more','Homeowner Share','Summer 2020 Noise','Avg. Noise',
'% Retail Employees'),omit.stat=c("LL","ser","f"))
```


A better explanation might be that one explanation won't fit every district, or put differently that there are large and unmeasured district level effects. To test this a fixed effects model was run using council districts. R's 'fixest' package was used, which includes robust standard errors and estimates the effects of district indirectly to avoid multicollinearity.[^4] The results are reported in Table 11 below. Now only income reaches significance at the 95% confidence level and home ownership at 90%. Additionally, the within R2 is very low, only rounding up to 1, while the overall R2 has risen to 56, suggesting 55% of variance is attributable to variance between districts that these models don't explain. It appears that these elections resist a one size fits all explanation of electoral success.

[^4]: Bergé L (2018). "Efficient estimation of maximum likelihood models with multiple fixed-effects: the R package FENmlm." CREA Discussion Papers.


```{r, table 11,echo=FALSE,warning=FALSE , results='asis'}
models <- list(
fe1 <- feols(vote_share ~ scale(log_income) + scale(nhb21p)| district, data = cluster_three),
fe2 <- feols(vote_share ~ scale(log_income) + scale(nhb21p) + scale(h21p) + scale(white_transplant_ratio) | district, data = cluster_three),
fe3 <- feols(vote_share ~ scale(log_income) + scale(nhb21p) + scale(h21p) + scale(white_transplant_ratio) + scale(cvap21bapp) + scale(hh21op)  | district, data = cluster_three),
fe4 <- feols(vote_share ~ scale(log_income) + scale(nhb21p) + scale(h21p) + scale(white_transplant_ratio) + scale(cvap21bapp) + scale(hh21op) + scale(summer_noise_complaints) +  scale(perc_retail) | district, data = cluster_three),
fe5 <- feols(vote_share ~ scale(log_income) + scale(nhb21p) + scale(h21p) + scale(white_transplant_ratio) + scale(cvap21bapp) + scale(hh21op) + scale(summer_noise_complaints) + scale(mean_noise) + scale(perc_retail) | district, data = cluster_three)
)

cm <- c('scale(log_income)'    = 'Log MHHI',
        'scale(nhb21p)'    = 'NH Black Share',
        'scale(h21p)' = 'Hispanic Share',
        'scale(white_transplant_ratio)' = 'White Transplant Share',
        'scale(cvap21bapp)' = '% BA or more',
        'scale(hh21op)' = 'Homeowner Share',
        'scale(summer_noise_complaints)' = 'Summer 2020 Noise',
        'scale(mean_noise)' = 'Avg. Noise',
        'scale(perc_retail)' = '% Retail Employees'
        )

modelsummary(
  models,
  stars = TRUE,
  gof_omit = 'Deviance|RMSE|AIC|BIC|FE|Std.Errors|R2 Within Adj.',
  output = 'html',
  coef_rename = cm)
```


## Demographic Clustering

Another approach to understanding the electoral dynamics of this progressive group is to once again turn to clustering. Since this progressive group seems to wins different groups in different districts, clustering the council districts and seeing which clusters tend to elect the progressive bloc can help to tease out electoral patterns. The same clustering process used on the voting data was followed on the same set of demographic predictors used in the regression models, plus some additional categories of ethnicity as well as NYPD arrest data, 311 data, and primary mode of transportation in commuting. All tests used to determine the proper number of predetermined clusters pointed clearly to five. Table 12 shows the results for the progressive cluster, cluster 3. Three of the five demographic clusters are represented by the progressives. Table 13, directly below, shows the variables that were the most influential in defining the clusters.

</br>


```{r, table 12,echo=FALSE,warning=FALSE , results='asis'}

demo_clusters%>%
  filter(new_cluster == 4)%>%
  select(candidate,demo_cluster)%>%
  arrange(desc(demo_cluster))%>%
  gt()%>%
  cols_label(candidate = 'Council Member', demo_cluster = 'Cluster')
```


</br>


```{r, table 13,echo=FALSE,warning=FALSE , results='asis'}
cluster_influence %>%
  gt() %>%
  tab_header(
    title = "Top Variables and Scores for Each Cluster",
    subtitle = "Ranked by Importance"
  ) %>%
  tab_style(
    style = list(
      cell_text(weight = "bold", align = "center")
    ),
    locations = cells_column_labels(everything())
  ) %>%
  fmt_missing(
    columns = everything(),
    missing_text = "-"
  )

```


</br>

The demographic heterogeneity of the districts sheds light on why the models above had low explanatory power. Income and broad racial categories are the only variables that all districts have in common. In the next several chapters each district and candidate will be explored in more detail, but for the time being Table 14 below shows models fit on each cluster. Model one corresponds to cluster one, etc. The R squared values have risen significantly, and different variables evidently affect the outcome of progressive candidates differently. In the whiter and wealthier cluster 1 it is indeed the poorer districts that voted for the progressive. In the poorer and more Black and Hispanic cluster 4 (where both DSA candidates won) income has less of an effect but white transplants clearly do.

</br>


```{r, table 14,echo=FALSE,warning=FALSE , results='asis'}
scaled_cluster_three <- cluster_three %>%
  mutate(across(c(log_income, nhb21p, h21p, white_transplant_ratio, cvap21bapp, hh21op, summer_noise_complaints, mean_noise, perc_retail), scale))

scaled_cluster_three_demo <- left_join(scaled_cluster_three,demo_clusters, by='district')
sctd0 <- scaled_cluster_three_demo%>%
  filter(demo_cluster == 0)
sctd2 <- scaled_cluster_three_demo%>%
  filter(demo_cluster == 2)
sctd4 <- scaled_cluster_three_demo%>%
  filter(demo_cluster == 4)
lm1 <- lm(vote_share ~ log_income + nhb21p + h21p + white_transplant_ratio + cvap21bapp + hh21op + summer_noise_complaints + mean_noise  + perc_retail , data = sctd0)
lm2 <- lm(vote_share ~ log_income + nhb21p + h21p + white_transplant_ratio + cvap21bapp + hh21op + summer_noise_complaints + mean_noise  + perc_retail , data = sctd2)
lm3 <- lm(vote_share ~ log_income + nhb21p + h21p + white_transplant_ratio + cvap21bapp + hh21op + summer_noise_complaints + mean_noise  + perc_retail , data = sctd4)
stargazer(lm1,lm2,lm3, type = 'html', out = 'models.html',covariate.labels=c('Log MHHI', 'NH Black Share','Hispanic Share','White Transplant Share',
'% BA or more','Homeowner Share','Summer 2020 Noise','Avg. Noise',
'% Retail Employees'),omit.stat=c("LL","ser","f"))
```


Table X below shows the results from a mixed-level model run on all the districts in which cluster 4 candidates were running. This model allows several of the variables to have their own slopes. This succinctly shows how a variable affect the candidates different across clusters holding other variables constant.

## Conclusion

This chapter has explored clustering the NYC City Council by the way they vote, the way they co-sponsor legislation, and the areas of the city that elect them. Voting and co-sponsorship pointed to a clearly defined group of members that vote in protest against mayoral budgets and mayoral appointments and frequently co-sponsor bills that protect workers, migrants, women, children, and LGBTQ New Yorkers. This legislative activity fits well with the idea of an egalitarian urban order defined by Weaver and discussed in Chapter 1. This legislative activity does not map well onto electoral demographics. The members won in four of the five types of districts clustered by their demographic makeup, they have found a winning formula in a variety of electoral circumstances. The coming chapters will look in more detail at the way these races were won, as well as a couple of narrow DSA losses in Central Brooklyn and Western Queens.